# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /home/dzuniga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dzuniga/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dzuniga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

In [10]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/dzuniga/anaconda3/lib/python3.7/si

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test)

In [17]:
reports = []
for i in range(y_pred.shape[1]):
    reports.append(classification_report(y_test.iloc[:, i].values.tolist(), list(y_pred[:, i]), output_dict=True))

In [28]:
for i in range(36):
    print('============')
    print(df.columns.values.tolist()[4:][i])
    print(reports[i]['0'])

related
{'precision': 0.310126582278481, 'recall': 0.12818835840418574, 'f1-score': 0.1813975011568718, 'support': 1529}
request
{'precision': 0.8373129576567973, 'recall': 0.9706587931352648, 'f1-score': 0.8990684556875481, 'support': 5419}
offer
{'precision': 0.9955731949320714, 'recall': 0.9998466963053809, 'f1-score': 0.9977053694355209, 'support': 6523}
aid_related
{'precision': 0.5916795069337443, 'recall': 0.8054535920293655, 'f1-score': 0.6822118587608261, 'support': 3814}
medical_help
{'precision': 0.9206153846153846, 'recall': 0.9920424403183024, 'f1-score': 0.954995212256623, 'support': 6032}
medical_products
{'precision': 0.9461032001224927, 'recall': 0.9967736731730924, 'f1-score': 0.9707776904948939, 'support': 6199}
search_and_rescue
{'precision': 0.9736721261288841, 'recall': 0.9971782411036213, 'f1-score': 0.985285006195787, 'support': 6379}
security
{'precision': 0.9815295374751947, 'recall': 0.9998445031876847, 'f1-score': 0.9906023725157911, 'support': 6431}
militar

In [257]:
y_pred = cv.predict(X_test)[2]

In [38]:
df.iloc[:, 4:].astype(int).sum().values.tolist()

[20298,
 4480,
 119,
 10878,
 2087,
 1314,
 724,
 471,
 860,
 0,
 1674,
 2930,
 2319,
 406,
 604,
 299,
 876,
 1196,
 3448,
 1705,
 1203,
 1335,
 534,
 159,
 283,
 120,
 309,
 1151,
 7302,
 2158,
 2448,
 282,
 2453,
 530,
 1376,
 5080]

In [255]:
y_pred = list(map(int, y_pred))

In [256]:
dict(zip(df.columns[4:], y_pred))

{'related': 1,
 'request': 0,
 'offer': 0,
 'aid_related': 1,
 'medical_help': 0,
 'medical_products': 0,
 'search_and_rescue': 0,
 'security': 0,
 'military': 0,
 'child_alone': 0,
 'water': 0,
 'food': 0,
 'shelter': 0,
 'clothing': 0,
 'money': 0,
 'missing_people': 0,
 'refugees': 0,
 'death': 0,
 'other_aid': 0,
 'infrastructure_related': 0,
 'transport': 0,
 'buildings': 0,
 'electricity': 0,
 'tools': 0,
 'hospitals': 0,
 'shops': 0,
 'aid_centers': 0,
 'other_infrastructure': 0,
 'weather_related': 0,
 'floods': 0,
 'storm': 0,
 'fire': 0,
 'earthquake': 0,
 'cold': 0,
 'other_weather': 0,
 'direct_report': 0}

### 6. Improve your model
Use grid search to find better parameters. 

In [201]:
parameters = {
    'clf__estimator__criterion' : ('gini', 'entropy'),
    #'clf__estimator__max_depth' : (None, 2, 5),
    #'clf__estimator__n_estimators' : (10, 20, 50),
    'clf__estimator__min_samples_leaf' : (1, 5, 10)
}

cv = GridSearchCV(pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [202]:
cv.fit(X_train, y_train)
cv.best_params_

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  >>> import numpy as np
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

...

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in vers

/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10
/home/dzuniga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  self.n_estimators = 10


{'clf__estimator__criterion': 'entropy',
 'clf__estimator__min_samples_leaf': 10}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.